# Step 4: PostProcess Result

In this part, we post process the optimizing results for evaluation and visualization.
At first, we prepare the following parameters:

| Parameter | Description | Value |
|--- |--- |--- |
|method | annealing method for QUBO problem |'dwave-sa': use the simulated annealer in ocean toolkit<br> 'dwave-qa': use the quantum annealer|
|raw_path| the path for the original molecule file| './molecule-data/117_ideal.mol2' in this example |
|data_path| the path for the processed molecule file| './qmu_117_ideal_data_latest.mol2' in this example |
|bucket | the s3 bucket to store your results | - |
|prefix | the name of the folder in your s3 bucket | - |
|task_id | the id for your quantum annealing task| '2b5a3b05-1a0e-443a-852c-4ec422a10e59' in this example |

Then we can run the post-process using **ResultParser** object for SA:

In [ ]:
from utility.MoleculeParser import MoleculeData
from utility.QMUQUBO import QMUQUBO
from utility.AnnealerOptimizer import Annealer
from utility.ResultProcess import ResultParser
import time

timestamp = time.strftime("%Y%m%d-%H")

In [18]:
method = "dwave-sa"
sa_param = {}
sa_param["raw_path"] = raw_path
sa_param["data_path"] = data_path

sa_process_result = ResultParser(method, **sa_param)
# print(f"{method} result is {sa_process_result.get_all_result()}")

local_time, _ , _, _= sa_process_result.get_time()

print(f"time for {method}: \n \
    local time is {local_time}")

INFO:root:_load_raw_result
INFO:root:load simulated annealer raw result
INFO:root:MoleculeData.load()
INFO:root:init mol data for final position
INFO:root:init mol data for raw position
INFO:root:_parse_model_info
INFO:root:_init_parameters
INFO:root:parse simulated annealer result
INFO:root:sa only has local_time!


time for dwave-sa: 
     local time is 175.73437476158142


In [19]:
sa_atom_pos_data = sa_process_result.generate_optimize_pts()
# save unfold file for visualization and parameters for experiment: 1. volume value 2. relative improvement
sa_process_result.save_mol_file(f"{timestamp}")


INFO:root:generate_optimize_pts()
INFO:numexpr.utils:NumExpr defaulting to 2 threads.
INFO:root:save_mol_file 20220216-06
INFO:root:finish save ./molecule-data/117_ideal_dwave-sa_20220216-06.mol2 and ./molecule-data/117_ideal_dwave-sa_20220216-06.json


['./molecule-data/117_ideal_dwave-sa_20220216-06.mol2',
 './molecule-data/117_ideal_dwave-sa_20220216-06.json']

In [20]:
sa_process_result.parameters

{'volume': {'optimize': 18.9423642473021,
  'initial': 18.549876807564765,
  'gain': 1.0211584930621898,
  'unfolding_results': ['x_15_7', 'x_14_8']}}

In the first block, we can see the **local time**
for SA is around 174 seconds. 
With the **generate_optimize_pts()** method, the final 3D 
points after unfolding will be generated and saved as json file and mol2 files. The last 
block shows the optimizing results which are also stored in json files. 
It shows that the optimized result gains 
1.0212x increase in volume. The value for **unfolding_results** indicates 
that the rotatable bond 15 should rotate $270^o$ ($360/8*(7-1)$) and 
the rotatable bond 14 should rotate $315^o$ ($360/8*(8-1)$).
At the same time, you can run the post-process for QA:

In [21]:
method = "dwave-qa"
qa_param = {}
qa_param["bucket"] = s3_bucket
qa_param["prefix"] = prefix
qa_param["task_id"] = qa_task_id
qa_param["raw_path"] = raw_path
qa_param["data_path"] = data_path

qa_process_result = ResultParser(method, **qa_param)
# print(f"{method} result is {qa_process_result.get_all_result()}")

local_time, task_time, total_time, access_time = qa_process_result.get_time()

print(f"time for {method}: \n \
    local time is {local_time},\n \
    task time is {task_time}, \n \
    qpu total time is {total_time}, \n \
    qpu access time is {access_time}")

INFO:root:_load_raw_result
INFO:root:load quantum annealer raw result
INFO:root:_read_result_obj
INFO:root:_read_result_obj: annealer-experiment/89a59fa9-c66e-482d-85e0-6dfce8c54454/qa_result.pickle
INFO:root:MoleculeData.load()
INFO:root:init mol data for final position
INFO:root:init mol data for raw position
INFO:root:_parse_model_info
INFO:root:_init_parameters
INFO:root:parse quantum annealer result
INFO:root:_read_result_obj
INFO:root:_read_result_obj: annealer-experiment/89a59fa9-c66e-482d-85e0-6dfce8c54454/results.json


time for dwave-qa: 
     local time is 7.57835578918457,
     task time is 1.89, 
     qpu total time is 0.142533, 
     qpu access time is 0.124129


we can see that there many types of time metrics for running QA.
This task has the **local time** of 7.7 s, which means the time between calling the api and 
getting the annealing result. The **task time** time is the metric from the json file in 
bucket. We can also see the **qpu total time** and **qpu access time** representing the 
actual time running in the QPU. Please refer to [Operation and Timing](https://docs.dwavesys.com/docs/latest/c_qpu_timing.html)
for details.

In [22]:
qa_atom_pos_data = qa_process_result.generate_optimize_pts()
# save unfold file for visualization and parameters for experiment: 1. volume value 2. relative improvement
qa_process_result.save_mol_file(f"{timestamp}")

INFO:root:generate_optimize_pts()
INFO:root:save_mol_file 20220216-06
INFO:root:finish save ./molecule-data/117_ideal_dwave-qa_20220216-06.mol2 and ./molecule-data/117_ideal_dwave-qa_20220216-06.json


['./molecule-data/117_ideal_dwave-qa_20220216-06.mol2',
 './molecule-data/117_ideal_dwave-qa_20220216-06.json']

In [23]:
qa_process_result.parameters

{'volume': {'optimize': 18.9423642473021,
  'initial': 18.549876807564765,
  'gain': 1.0211584930621898,
  'unfolding_results': ['x_15_7', 'x_14_8']}}

In same way, the optimized results are translated the 3D points and saved 
as local json and mol2 files. The result indicates that QA gains 
1.021x increase in 
volume.

Finally, We can open folders for the optimized results:

![optimize-results](../../../images/optimize-results.png)

 <center>Optimize Results</center>

We can the json result and mol2 file of SA and QA are 
stored in this place. If we carry out more 
experiments, more results with time stamp are 
stored incrementally. 
For visualization, 
we can upload the 
result **117_ideal_dwave-qa_20220216-05.mol2** 
into 
[online viewer tool](https://www.rcsb.org/3d-view) 
to see the result:

![visual](../../../images/visualization.png)

 <center>Visualization</center>